# Read DVH File Tests

## Setup

### Imports

In [140]:
from typing import Callable, List

import re
import logging
import pprint
from pathlib import Path

#### Packages

In [141]:
import numpy as np
import pandas as pd

#### Local Imports

In [142]:
from buffered_iterator import BufferedIterator

import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, Section, ProcessingMethods
#import read_dvh_file

### Logging

In [143]:
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
logger = logging.getLogger('read_dvh.file')
logger.setLevel(logging.DEBUG)

### Helper Functions

#### Function to compare two dictionaries

In [144]:
def compare_dict(dict1, dict2, one_line=True):
    one_line_template = '{key:32s}:\t{item1:32s}\t{item2:32s}'
    stacked_template = '{key}:\n\t{item1}\n\t{item2}'
    
    keys_1 = set(dict1.keys())
    keys_2 = set(dict2.keys())
    all_keys = keys_1 | keys_2
    for key in all_keys:
        item1 = pprint.pformat(str(dict1.get(key, '')))
        item2 = pprint.pformat(str(dict2.get(key, '')))
        if one_line:
            cmp_str = one_line_template.format(key=str(key), 
                                               item1=item1, 
                                               item2=item2)
        else:
            cmp_str = stacked_template.format(key=str(key), 
                                              item1=item1, 
                                              item2=item2)
        print(cmp_str)

## Data

### Source

In [145]:
test_source = [
    'Patient Name         : ____, ____',
    'Patient ID           : 1234567',
    'Comment              : DVHs for multiple plans and plan sums',
    'Date                 : Friday, January 17, 2020 09:45:07',
    'Exported by          : gsal',
    'Type                 : Cumulative Dose Volume Histogram',
    ('Description          : The cumulative DVH displays the '
    'percentage (relative)'),
    ('                       or volume (absolute) of structures '
    'that receive a dose'),
    '                      equal to or greater than a given dose.',
    ''
    'Plan sum: Plan Sum',
    'Course: PLAN SUM',
    'Prescribed dose [cGy]: not defined',
    '% for dose (%): not defined',
    ''
    'Plan: PARR',
    'Course: C1',
    'Plan Status: Treatment Approved Thursday, January 02, '
    '2020 12:55:56 by gsal',
    'Prescribed dose [cGy]: 5000.0',
    '% for dose (%): 100.0',
    ''
    'Structure: PRV5 SpinalCanal',
    'Approval Status: Approved',
    'Plan: Plan Sum',
    'Course: PLAN SUM',
    'Volume [cm³]: 121.5',
    'Dose Cover.[%]: 100.0',
    'Sampling Cover.[%]: 100.1',
    'Min Dose [cGy]: 36.7',
    'Max Dose [cGy]: 3670.1',
    'Mean Dose [cGy]: 891.9',
    'Modal Dose [cGy]: 44.5',
    'Median Dose [cGy]: 863.2',
    'STD [cGy]: 621.9',
    'NDR: ',
    'Equiv. Sphere Diam. [cm]: 6.1',
    'Conformity Index: N/A',
    'Gradient Measure [cm]: N/A',
    ''
    'Dose [cGy] Ratio of Total Structure Volume [%]',
    '         0                       100',
    '         1                       100',
    '         2                       100',
    '         3                       100',
    '         4                       100',
    '         5                       100',
    '      3667              4.23876e-005',
    '      3668              2.87336e-005',
    '      3669              1.50797e-005',
    '      3670               1.4257e-006',
    '',
    'Structure: PTV 50',
    'Approval Status: Approved',
    'Plan: Plan Sum',
    'Course: PLAN SUM',
    'Volume [cm³]: 363.6',
    'Dose Cover.[%]: 100.0',
    'Sampling Cover.[%]: 100.0',
    'Min Dose [cGy]: 3985.9',
    'Max Dose [cGy]: 5442.0',
    'Mean Dose [cGy]: 5144.5',
    'Modal Dose [cGy]: 5177.3',
    'Median Dose [cGy]: 5166.9',
    'STD [cGy]: 131.9',
    'NDR: ',
    'Equiv. Sphere Diam. [cm]: 8.9',
    'Conformity Index: N/A',
    'Gradient Measure [cm]: N/A',
    '',
    'Dose [cGy] Ratio of Total Structure Volume [%]',
    '         0                       100',
    '         1                       100',
    '         2                       100',
    '         3                       100',
    '         4                       100',
    '         5                       100',
    '      5437               9.4777e-005',
    '      5438              6.35607e-005',
    '      5439              3.62425e-005',
    '      5440              1.82336e-005',
    '      5441              9.15003e-006',
    '      5442               6.6481e-008'
                ]


### Results
#### DVH Info

In [146]:
DVH_Info = {
    'Patient Name': '____, ____',
    'Patient ID': '1234567',
    'Comment': 'DVHs for multiple plans and plan sums',
    'Date': 'Friday, January 17, 2020 09:45:07',
    'Exported by': 'gsal',
    'Type': 'Cumulative Dose Volume Histogram',
    'Description': ('The cumulative DVH displays the '
                    'percentage (relative) or volume '
                    '(absolute) of structures that receive a '
                    'dose equal to or greater than a '
                    'given dose.')
    }
DVH_Info

{'Patient Name': '____, ____',
 'Patient ID': '1234567',
 'Comment': 'DVHs for multiple plans and plan sums',
 'Date': 'Friday, January 17, 2020 09:45:07',
 'Exported by': 'gsal',
 'Type': 'Cumulative Dose Volume Histogram',
 'Description': 'The cumulative DVH displays the percentage (relative) or volume (absolute) of structures that receive a dose equal to or greater than a given dose.'}

#### Plan Info

In [147]:
Plan_Info = {
    'Plan Sum': {'Plan sum': 'Plan Sum',
                    'Plan': 'Plan Sum',
                    'Course': 'PLAN SUM',
                    'Prescribed dose': '',
                    'Prescribed dose unit': '',
                    '% for dose (%)': 'not defined'},
    'PARR': {'Plan': 'PARR',
                'Course': 'C1',
                'Plan Status': 'Treatment Approved',
                'Approved on': 'Thursday, January 02, 2020 12:55:56',
                'Approved by': 'gsal',
                'Prescribed dose': 5000.0,
                'Prescribed dose unit': 'cGy',
                '% for dose (%)': 100.0}
    }
Plan_Info

{'Plan Sum': {'Plan sum': 'Plan Sum',
  'Plan': 'Plan Sum',
  'Course': 'PLAN SUM',
  'Prescribed dose': '',
  'Prescribed dose unit': '',
  '% for dose (%)': 'not defined'},
 'PARR': {'Plan': 'PARR',
  'Course': 'C1',
  'Plan Status': 'Treatment Approved',
  'Approved on': 'Thursday, January 02, 2020 12:55:56',
  'Approved by': 'gsal',
  'Prescribed dose': 5000.0,
  'Prescribed dose unit': 'cGy',
  '% for dose (%)': 100.0}}

#### Structures

In [148]:
structures = pd.DataFrame({
    'Course': ['PLAN SUM', 'PLAN SUM',],
    'Plan': ['Plan Sum', 'Plan Sum'],
    'Structure': ['PRV5 SpinalCanal', 'PTV 50',],
    'Approval Status': ['Approved', 'Approved'],
    'Volume [cc]': [121.5, 363.6,],
    'Dose Cover.[%]': [100.0, 100.0,],
    'Sampling Cover.[%]': [100.1, 100.0,],
    'Min Dose [cGy]': [36.7, 3985.9,],
    'Max Dose [cGy]': [3670.1, 5442.0,],
    'Mean Dose [cGy]': [891.9, 5144.5,],
    'Modal Dose [cGy]': [44.5, 5177.3,],
    'Median Dose [cGy]': [863.2, 5166.9,],
    'STD [cGy]': [621.9, 131.9,],
    'NDR': ['', ''],
    'Equiv. Sphere Diam. [cm]': [6.1, 8.9],
    'Conformity Index': ['N/A', 'N/A'],
    'Gradient Measure [cm]': ['N/A', 'N/A']
    })
structures.set_index(['Course', 'Plan', 'Structure'], drop=False,
                        inplace=True)
structures = structures.T
structures

Course                            PLAN SUM          
Plan                              Plan Sum          
Structure                 PRV5 SpinalCanal    PTV 50
Course                            PLAN SUM  PLAN SUM
Plan                              Plan Sum  Plan Sum
Structure                 PRV5 SpinalCanal    PTV 50
Approval Status                   Approved  Approved
Volume [cc]                          121.5     363.6
Dose Cover.[%]                       100.0     100.0
Sampling Cover.[%]                   100.1     100.0
Min Dose [cGy]                        36.7    3985.9
Max Dose [cGy]                      3670.1    5442.0
Mean Dose [cGy]                      891.9    5144.5
Modal Dose [cGy]                      44.5    5177.3
Median Dose [cGy]                    863.2    5166.9
STD [cGy]                            621.9     131.9
NDR                                                 
Equiv. Sphere Diam. [cm]               6.1       8.9
Conformity Index                       N/A       N/A
Gradient Measure [cm]                  N/A       N/A

#### DVH

In [149]:
dvh = pd.DataFrame({
    'Course': ['PLAN SUM', 'PLAN SUM', 'PLAN SUM', 'PLAN SUM'],
    'Plan': ['Plan Sum', 'Plan Sum', 'Plan Sum', 'Plan Sum'],
    'Structure': ['PRV5 SpinalCanal', 'PRV5 SpinalCanal',
                    'PTV 50', 'PTV 50'],
    'Data': ['Dose [cGy]', 'Ratio of Total Structure Volume [%]',
                'Dose [cGy]', 'Ratio of Total Structure Volume [%]'],
    0: [0, 100, 0, 100],
    1: [1, 100, 1, 100],
    2: [2, 100, 2, 100],
    3: [3, 100, 3, 100],
    4: [4, 100, 4, 100],
    5: [5, 100, 5, 100],
    6: [3667, 4.23876e-005, 5437, 9.4777e-005],
    7: [3668, 2.87336e-005, 5438, 6.35607e-005],
    8: [3669, 1.50797e-005, 5439, 3.62425e-005],
    9: [3670, 1.4257e-006, 5440, 1.82336e-005],
    10: [np.nan, np.nan, 5441, 9.15003e-006],
    11: [np.nan, np.nan, 5442, 6.6481e-008]
    })
dvh.set_index(['Course', 'Plan', 'Structure', 'Data'],
                inplace=True)
dvh = dvh.T
dvh

Course            PLAN SUM                                                 \
Plan              Plan Sum                                                  
Structure PRV5 SpinalCanal                                         PTV 50   
Data            Dose [cGy] Ratio of Total Structure Volume [%] Dose [cGy]   
0                      0.0                          100.000000        0.0   
1                      1.0                          100.000000        1.0   
2                      2.0                          100.000000        2.0   
3                      3.0                          100.000000        3.0   
4                      4.0                          100.000000        4.0   
5                      5.0                          100.000000        5.0   
6                   3667.0                            0.000042     5437.0   
7                   3668.0                            0.000029     5438.0   
8                   3669.0                            0.000015     5439.0   
9                   3670.0                            0.000001     5440.0   
10                     NaN                                 NaN     5441.0   
11                     NaN                                 NaN     5442.0   

Course                                         
Plan                                           
Structure                                      
Data      Ratio of Total Structure Volume [%]  
0                                1.000000e+02  
1                                1.000000e+02  
2                                1.000000e+02  
3                                1.000000e+02  
4                                1.000000e+02  
5                                1.000000e+02  
6                                9.477700e-05  
7                                6.356070e-05  
8                                3.624250e-05  
9                                1.823360e-05  
10                               9.150030e-06  
11                               6.648100e-08

#### Context

In [150]:
context = {
    'File Name': 'Test_DVH_Sections.txt',
    'File Path': Path.cwd() / 'Text Files' / 'Test_DVH_Sections.txt',
    'Line Count': 0
    }
context

{'File Name': 'Test_DVH_Sections.txt',
 'File Path': WindowsPath("c:/Users/smoke/OneDrive - Queen's University/Python/Projects/sectionary package/Text Files/Test_DVH_Sections.txt"),
 'Line Count': 0}

## Section Definitions

### Line Parsing Functions

#### Date Rule

In [151]:
def make_date_parse_rule() -> Rule:
    def date_parse(line: str) -> tp.ProcessedList:
        '''If Date,don't split beyond first :.'''
        parsed_line = line.split(':', maxsplit=1)
        return parsed_line

    date_rule = Rule('Date', location='START', name='date_rule',
                        pass_method=date_parse, fail_method='None')
    return date_rule


#### Approved Status

In [152]:
def make_approved_status_rule() -> Rule:
    '''If Treatment Approved, Split "Plan Status" into 3 lines:
        Plan Status
        Approved on
        Approved by
        '''
    def approved_status_parse(line, event) -> tp.ProcessedList:
        '''If Treatment Approved, Split "Plan Status" into 3 lines:

        Return three rows for a line containing "Treatment Approved"
            Prescribed dose [unit]: dose
        Gives:
            [['Plan Status', 'Treatment Approved'],
             ['Approved on', date],
             ['Approved by', person]
        '''
        idx1 = line.find(event.test_value)
        idx2 = idx1 + len(event.test_value)
        idx3 = line.find(' by')
        idx4 = idx3 + 4
        parsed_lines = [
            ['Plan Status', line[idx1:idx2]],
            ['Approved on', line[idx2+1:idx3]],
            ['Approved by', line[idx4:]]
            ]
        for line in parsed_lines:
            yield line

    approved_status_rule = Rule('Treatment Approved', location='IN',
                                   pass_method=approved_status_parse,
                                   fail_method='None',
                                   name='approved_status_rule')
    return approved_status_rule


#### Prescribed Dose Rule

In [153]:
def make_prescribed_dose_rule() -> Rule:
    def parse_prescribed_dose(line, event) -> tp.ProcessedList:
        '''Split "Prescribed dose [cGy]" into 2 lines.

        Return two rows for a line containing:
            Prescribed dose [unit]: dose
        Gives:
            [['Prescribed dose', 'dose'],
            ['Prescribed dose unit', 'unit']],
        The line:
            Prescribed dose [unit]: not defined
        Results in:
            [['Prescribed dose', '5000.0'],
             ['Prescribed dose unit', 'cGy']]
        '''
        match_results = event.test_value.groupdict()
        if match_results['dose'] == 'not defined':
            match_results['dose'] = ''
            match_results['unit'] = ''

        parsed_lines = [
            ['Prescribed dose', match_results['dose']],
            ['Prescribed dose unit', match_results['unit']]
            ]
        for line in parsed_lines:
            yield line

    prescribed_dose_pattern = (
        r'^Prescribed dose\s*'  # Begins with Prescribed dose
        r'\['                   # Unit start delimiter
        r'(?P<unit>[A-Za-z]+)'  # unit group: text surrounded by []
        r'\]'                   # Unit end delimiter
        r'\s*:\s*'              # Dose delimiter with possible whitespace
        r'(?P<dose>[0-9.]+'     # dose group Number
        r'|not defined)'        #"not defined" alternative
        r'[\s\r\n]*'            # drop trailing whitespace
        r'$'                    # end of string
        )
    re_pattern = re.compile(prescribed_dose_pattern)
    dose_rule = Rule(sentinel=re_pattern, name='prescribed_dose_rule',
                        pass_method= parse_prescribed_dose, fail_method='None')
    return dose_rule


#### Default CSV Parser

In [154]:
def make_default_csv_parser() -> Callable:
    default_csv = tp.define_csv_parser('dvh_info', delimiter=':',
                                       skipinitialspace=True)
    return default_csv


### Post Processing Methods

#### fix_structure_names

In [155]:
def fix_structure_names(line: List[str]) -> List[str]:
    '''If Structure name starts with "=", add "'" to start of name.
    '''
    if len(line) == 2:
        if 'Structure' in line[0]:
            structure_name = line[1]
            if structure_name.startswith('='):
                structure_name = "'" + structure_name
                line[1] = structure_name
    return line


#### Line Processing

In [156]:
def to_plan_info_dict(plan_info_dict_list):
    '''Combine Plan Info dictionaries into dictionary of dictionaries.
    '''
    output_dict = dict()
    for plan_info_dict in plan_info_dict_list:
        if len(plan_info_dict) == 0:
            continue
        plan_name = plan_info_dict.get('Plan')
        if not plan_name:
            plan_name = plan_info_dict.get('Plan sum')
            if not plan_name:
                plan_name = 'Plan'
            plan_info_dict['Plan'] = plan_name
        output_dict[plan_name] = plan_info_dict
    return output_dict


#### to_structure_data_tuple

In [157]:
def to_structure_data_tuple(structure_data_list):
    '''Combine Structure and DVH data.
    '''
    structures_dict = dict()
    dvh_data_list = list()
    for structure_data_set in structure_data_list:
        structure_data = structure_data_set['Structure']
        dvh_data = structure_data_set['DVH']
        plan_name = structure_data['Plan']
        course_id = structure_data['Course']
        structure_id = structure_data['Structure']
        logger.info(f'Reading DVH data for: {structure_id}.')
        indx = (course_id, plan_name, structure_id)
        structures_dict[indx] = structure_data
        data_columns = list(dvh_data.columns)
        indx_d = [indx + (d,) for d in data_columns]
        indx_names = ['Course', 'Plan', 'Structure', 'Data']
        index = pd.MultiIndex.from_tuples(indx_d, names=indx_names)
        dvh_data.columns = index
        dvh_data_list.append(dvh_data)
    structures_df = pd.DataFrame(structures_dict)
    dvh_df = pd.concat(dvh_data_list, axis='columns')
    return (structures_df, dvh_df)


### Reader definitions

In [158]:
default_parser = tp.define_csv_parser('dvh_info', delimiter=':',
                                      skipinitialspace=True)


In [159]:
dvh_info_reader = ProcessingMethods([
    tp.clean_ascii_text,
    RuleSet([make_date_parse_rule()], default=default_parser),
    tp.trim_items,
    tp.drop_blanks,
    tp.merge_continued_rows
    ])


In [160]:
plan_info_reader = ProcessingMethods([
    tp.clean_ascii_text,
    RuleSet([make_prescribed_dose_rule(), make_approved_status_rule()],
               default=default_parser),
    tp.trim_items,
    tp.drop_blanks,
    tp.convert_numbers
    ])


In [161]:
structure_info_reader = ProcessingMethods([
    tp.clean_ascii_text,
    default_parser,
    tp.trim_items,
    tp.drop_blanks,
    tp.convert_numbers,
    fix_structure_names
    ])


In [162]:
dvh_data_reader = ProcessingMethods([
    tp.clean_ascii_text,
    tp.define_fixed_width_parser(widths=10),
    tp.trim_items,
    tp.drop_blanks,
    tp.convert_numbers
    ])

### SectionBreak definitions

In [163]:
plan_info_start = SectionBreak(
    name='Start of Plan Info',
    sentinel=['Plan:', 'Plan sum:'],
    break_offset='Before'
    )


In [164]:
plan_info_end = SectionBreak(
    name='End of Plan Info',
    sentinel='% for dose (%):',
    break_offset='After'
    )


In [165]:
structure_info_start = SectionBreak(
    name='Start of Structure Info',
    sentinel='Structure:',
    break_offset='Before'
    )


In [166]:
structure_info_end = SectionBreak(
    name='End of Structure Info',
    sentinel='Gradient Measure',
    break_offset='After'
    )


In [167]:
dvh_data_start = SectionBreak(
    name='Start of DVH Data',
    sentinel='Ratio of Total Structure Volume',
    break_offset='Before'
    )

### Section definitions

In [168]:
dvh_info_section = Section(
    name='DVH Info',
    start_section=None,
    end_section=plan_info_start,
    processor=dvh_info_reader,
    assemble=tp.to_dict
    )


In [169]:
plan_info_section = Section(
    name='Plan Info',
    start_section=None,
    end_section=plan_info_end,
    processor=plan_info_reader,
    assemble=tp.to_dict
    )


In [170]:
plan_info_group = Section(
    name='Plan Info Group',
    start_section=plan_info_start,
    end_section=structure_info_start,
    processor=plan_info_section,
    assemble=to_plan_info_dict
    )


In [171]:
structure_info_section = Section(
    name='Structure',
    start_section=structure_info_start,
    end_section=structure_info_end,
    processor=structure_info_reader,
    assemble=tp.to_dict
    )


In [172]:
dvh_data_section = Section(
    name='DVH',
    start_section=dvh_data_start,
    end_section=structure_info_start,
    processor=dvh_data_reader)


In [173]:
dvh_group_section = Section(
    name='DVH Groups',
    start_section=structure_info_start,
    processor=[[structure_info_section, dvh_data_section]])

## Testing

### DVH_Info

In [174]:
dvh_info_section = dvh_info_section
source = BufferedIterator(test_source)
test_output = dvh_info_section.read(source, context=context)
compare_dict(test_output, DVH_Info, one_line=False)

Date:
	'Friday, January 17, 2020 09:45:07'
	'Friday, January 17, 2020 09:45:07'
Comment:
	'DVHs for multiple plans and plan sums'
	'DVHs for multiple plans and plan sums'
Patient Name:
	'____, ____'
	'____, ____'
Patient ID:
	'1234567'
	'1234567'
Type:
	'Cumulative Dose Volume Histogram'
	'Cumulative Dose Volume Histogram'
Exported by:
	'gsal'
	'gsal'
Description:
	('The cumulative DVH displays the percentage (relative) or volume (absolute) '
 'of structures that receive a dose equal to or greater than a given dose.')
	('The cumulative DVH displays the percentage (relative) or volume (absolute) '
 'of structures that receive a dose equal to or greater than a given dose.')


### Plan Info

In [175]:
source = BufferedIterator(test_source)
plan_info = plan_info_group.read(source, context=context)
pprint.pprint(plan_info)
#compare_dict(plan_info, Plan_Info, one_line=False)

{'PARR': {'% for dose (%)': 100.0,
          'Approved by': 'gsal',
          'Approved on': 'Thursday, January 02, 2020 12:55:56',
          'Course': 'C1',
          'Plan': 'PARR',
          'Plan Status': 'Treatment Approved',
          'Prescribed dose': 5000.0,
          'Prescribed dose unit': 'cGy'},
 'Plan Sum': {'% for dose (%)': 'not defined',
              'Course': 'PLAN SUM',
              'Plan': 'Plan Sum',
              'Plan sum': 'Plan Sum',
              'Prescribed dose': '',
              'Prescribed dose unit': ''}}



**Expected:**

In [176]:
pprint.pprint(Plan_Info)

{'PARR': {'% for dose (%)': 100.0,
          'Approved by': 'gsal',
          'Approved on': 'Thursday, January 02, 2020 12:55:56',
          'Course': 'C1',
          'Plan': 'PARR',
          'Plan Status': 'Treatment Approved',
          'Prescribed dose': 5000.0,
          'Prescribed dose unit': 'cGy'},
 'Plan Sum': {'% for dose (%)': 'not defined',
              'Course': 'PLAN SUM',
              'Plan': 'Plan Sum',
              'Plan sum': 'Plan Sum',
              'Prescribed dose': '',
              'Prescribed dose unit': ''}}


### DVH

In [179]:
source = BufferedIterator(test_source)
dvh_df = dvh_data_section.read(source, context=context)
#dvh_df.fillna(0, inplace=True)

#dvh.fillna(0, inplace=True)
#compare_dict(dvh_df.to_dict(), dvh.to_dict(), one_line=False)
dvh_df

[['Dose [cGy]', 'Ratio of Total Structure Volume [%]'],
 [0.0, 100.0],
 [1.0, 100.0],
 [2.0, 100.0],
 [3.0, 100.0],
 [4.0, 100.0],
 [5.0, 100.0],
 [3667.0, 4.23876e-05],
 [3668.0, 2.87336e-05],
 [3669.0, 1.50797e-05],
 [3670.0, 1.4257e-06]]

In [181]:
source = BufferedIterator(test_source)
structure_data_list = dvh_group_section.read(source, context=context)
structure_data_list


[{'Structure': {'Structure': 'PRV5 SpinalCanal',
   'Approval Status': 'Approved',
   'Plan': 'Plan Sum',
   'Course': 'PLAN SUM',
   'Volume [cc]': 121.5,
   'Dose Cover.[%]': 100.0,
   'Sampling Cover.[%]': 100.1,
   'Min Dose [cGy]': 36.7,
   'Max Dose [cGy]': 3670.1,
   'Mean Dose [cGy]': 891.9,
   'Modal Dose [cGy]': 44.5,
   'Median Dose [cGy]': 863.2,
   'STD [cGy]': 621.9,
   'NDR': '',
   'Equiv. Sphere Diam. [cm]': 6.1,
   'Conformity Index': 'N/A',
   'Gradient Measure [cm]': 'N/A'},
  'DVH': [['Dose [cGy]', 'Ratio of Total Structure Volume [%]'],
   [0.0, 100.0],
   [1.0, 100.0],
   [2.0, 100.0],
   [3.0, 100.0],
   [4.0, 100.0],
   [5.0, 100.0],
   [3667.0, 4.23876e-05],
   [3668.0, 2.87336e-05],
   [3669.0, 1.50797e-05],
   [3670.0, 1.4257e-06]]},
 {'Structure': {'Structure': 'PTV 50',
   'Approval Status': 'Approved',
   'Plan': 'Plan Sum',
   'Course': 'PLAN SUM',
   'Volume [cc]': 363.6,
   'Dose Cover.[%]': 100.0,
   'Sampling Cover.[%]': 100.0,
   'Min Dose [cGy]': 3

In [182]:

structures_list = list()
dvh_data_list = list()
for structure_data_set in structure_data_list:
    structure_data = structure_data_set['Structure']
    dvh_data = tp.to_dataframe(structure_data_set['DVH'])
    
    plan_name = structure_data['Plan']
    course_id = structure_data['Course']
    structure_id = structure_data['Structure']
    

    indx = (course_id, plan_name, structure_id)
    structures_list.append(pd.Series(structure_data))
    
    dvh_data['Plan'] = structure_data['Plan']
    dvh_data['Course'] = structure_data['Course']
    dvh_data['Structure'] = structure_data['Structure']
    
    data_columns = list(dvh_data.columns)
    indx_d = [indx + (d,) for d in data_columns]
    indx_names = ['Course', 'Plan', 'Structure', 'Data']
    index = pd.MultiIndex.from_tuples(indx_d, names=indx_names)
    dvh_data.columns = index
    dvh_data_list.append(dvh_data)
#structures_df = pd.DataFrame(structures_dict)
dvh_df = pd.concat(dvh_data_list, axis='columns')



read_dvh.file        - INFO: Reading DVH data for: PRV5 SpinalCanal.


AttributeError: 'list' object has no attribute 'columns'

In [ ]:
source = BufferedIterator(test_source)
structures_df = dvh_group_section.read(source, context=context)
structures_df


[{'Structure': 'PRV5 SpinalCanal',
  'Approval Status': 'Approved',
  'Plan': 'Plan Sum',
  'Course': 'PLAN SUM',
  'Volume [cc]': 121.5,
  'Dose Cover.[%]': 100.0,
  'Sampling Cover.[%]': 100.1,
  'Min Dose [cGy]': 36.7,
  'Max Dose [cGy]': 3670.1,
  'Mean Dose [cGy]': 891.9,
  'Modal Dose [cGy]': 44.5,
  'Median Dose [cGy]': 863.2,
  'STD [cGy]': 621.9,
  'NDR': '',
  'Equiv. Sphere Diam. [cm]': 6.1,
  'Conformity Index': 'N/A',
  'Gradient Measure [cm]': 'N/A'},
 {'Structure': 'PTV 50',
  'Approval Status': 'Approved',
  'Plan': 'Plan Sum',
  'Course': 'PLAN SUM',
  'Volume [cc]': 363.6,
  'Dose Cover.[%]': 100.0,
  'Sampling Cover.[%]': 100.0,
  'Min Dose [cGy]': 3985.9,
  'Max Dose [cGy]': 5442.0,
  'Mean Dose [cGy]': 5144.5,
  'Modal Dose [cGy]': 5177.3,
  'Median Dose [cGy]': 5166.9,
  'STD [cGy]': 131.9,
  'NDR': '',
  'Equiv. Sphere Diam. [cm]': 8.9,
  'Conformity Index': 'N/A',
  'Gradient Measure [cm]': 'N/A'}]

In [ ]:
source = BufferedIterator(test_source)
structures_df, dvh_df = dvh_group_section.read(source, context=context)
dvh_df.fillna(0, inplace=True)

dvh.fillna(0, inplace=True)
compare_dict(dvh_df.to_dict(), dvh.to_dict(), one_line=False)


AttributeError: 'dict' object has no attribute 'fillna'

### Structures

In [124]:
dvh_group_section = dvh_group_section
# scan_section
source = BufferedIterator(test_source)
structures_df = structure_info_section.read(source, context=context)
#compare_dict(structures_df.to_dict(), structures.to_dict(), one_line=False)
structures_df

{'Structure': 'PRV5 SpinalCanal',
 'Approval Status': 'Approved',
 'Plan': 'Plan Sum',
 'Course': 'PLAN SUM',
 'Volume [cc]': 121.5,
 'Dose Cover.[%]': 100.0,
 'Sampling Cover.[%]': 100.1,
 'Min Dose [cGy]': 36.7,
 'Max Dose [cGy]': 3670.1,
 'Mean Dose [cGy]': 891.9,
 'Modal Dose [cGy]': 44.5,
 'Median Dose [cGy]': 863.2,
 'STD [cGy]': 621.9,
 'NDR': '',
 'Equiv. Sphere Diam. [cm]': 6.1,
 'Conformity Index': 'N/A',
 'Gradient Measure [cm]': 'N/A'}

In [ ]:
dvh_group_section = dvh_group_section
# scan_section
source = BufferedIterator(test_source)
structures_df, dvh_df = dvh_group_section.read(source, context=context)
#compare_dict(structures_df.to_dict(), structures.to_dict(), one_line=False)
structures_df